# Train Code of Team Stay Safe

## Part 1: Read Image into dataset

### 1. Read into normal dataset (Version 1:train and test data is splited in folder)

In [1]:
# import torchvision
# import torch
# from torchvision import transforms

# # check if cuda is available for boost
# cuda = torch.cuda.is_available()

# # mean and std for normalization
# mean, std = 0.1307, 0.3081

# # path of the train and test data folder
# train_path = 'train'
# test_path = 'test'

# # read train dataset with transformation
# train_dataset = torchvision.datasets.ImageFolder(
#     root=train_path,
#     transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
#                                                                   transforms.Resize((28,28)),
#                                                                   transforms.ToTensor(),
#                                                                   transforms.Normalize((mean,), (std,))
#                                                                   ]))

# print(len(train_dataset))

# # read test data with transformation
# test_dataset = torchvision.datasets.ImageFolder(
#     root=test_path,
#     transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
#                                                                   transforms.Resize((28,28)),
#                                                                   transforms.ToTensor(),
#                                                                   transforms.Normalize((mean,), (std,))
#                                                                   ])
# )

# # Set up data loaders
# batch_size = 256
# kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

### 1. Read into normal dataset (Version 2: only one folder contain all dataset)

In [2]:
import torchvision
import torch
from torchvision import transforms

# check if cuda is available for boost
cuda = torch.cuda.is_available()

# mean and std for normalization
mean, std = 0.1307, 0.3081

# path of the fulldataset folder
folder_path = 'data'

# read train dataset with transformation
full_dataset = torchvision.datasets.ImageFolder(
    root=folder_path,
    transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                                  transforms.Resize((28,28)),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((mean,), (std,))
                                                                  ]))

# train test split
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dataset = torch.utils.data.Subset(full_dataset, range(train_size))
test_dataset = torch.utils.data.Subset(full_dataset, range(train_size, len(full_dataset)))

# Set up data loaders
batch_size = 256
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

### 2. Triplet model dataset

Create Triplet dataset generator class

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler

class Triplet_dataset(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """

    def __init__(self, original_dataset, is_train, transform=None):
        self.original_dataset = original_dataset
        self.train = is_train
        self.transform = transform

        if self.train:
            self.train_labels = [] #create list to store labels
            for data in self.original_dataset: #iterate and initialize list
                self.train_labels.append(data[1])
            self.train_labels = np.array(self.train_labels) # list to numpy array
            self.train_labels = torch.from_numpy(self.train_labels) # numpy array to tensor
            self.labels_set = set(self.train_labels.numpy())
            self.label_to_indices = {label: np.where(self.train_labels.numpy() == label)[0]
                                     for label in self.labels_set}

        else:
            
            self.test_labels = [] #create list to store labels
            for data in self.original_dataset: #iterate and initialize list
                self.test_labels.append(data[1])
            self.test_labels = np.array(self.test_labels) # list to numpy array
            self.test_labels = torch.from_numpy(self.test_labels) # numpy array to tensor
            self.labels_set = set(self.test_labels.numpy())
            self.label_to_indices = {label: np.where(self.test_labels.numpy() == label)[0]
                                     for label in self.labels_set}

            random_state = np.random.RandomState(29)

            triplets = [[i,
                         random_state.choice(self.label_to_indices[self.test_labels[i].item()]),
                         random_state.choice(self.label_to_indices[
                                                 np.random.choice(
                                                     list(self.labels_set - set([self.test_labels[i].item()]))
                                                 )
                                             ])
                         ]
                        for i in range(len(self.original_dataset))]
            self.test_triplets = triplets

    def __getitem__(self, index):
        if self.train:
            img1_tuple, label1 = self.original_dataset.imgs[index], self.train_labels[index].item()
            positive_index = index
            while positive_index == index:
                positive_index = np.random.choice(self.label_to_indices[label1])
            negative_label = np.random.choice(list(self.labels_set - set([label1])))
            negative_index = np.random.choice(self.label_to_indices[negative_label])
            img2_tuple = self.original_dataset.imgs[positive_index]
            img3_tuple = self.original_dataset.imgs[negative_index]
        else:
            img1_tuple = self.original_dataset.imgs[self.test_triplets[index][0]]
            img2_tuple = self.original_dataset.imgs[self.test_triplets[index][1]]
            img3_tuple = self.original_dataset.imgs[self.test_triplets[index][2]]
            
        img1 = Image.open(img1_tuple[0])
        img1 = img1.convert("L")
        img2 = Image.open(img2_tuple[0])
        img2 = img1.convert("L")
        img3 = Image.open(img3_tuple[0])
        img3 = img1.convert("L")

        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)
            
        return (img1, img2, img3), []

    def __len__(self):
        return len(self.original_dataset)

Generate Siamese dataset

In [4]:
Triplet_train_dataset = Triplet_dataset(original_dataset=train_dataset.dataset,
                                        is_train = True,
                                        transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                                      transforms.Resize((28,28)),
                                                                      transforms.ToTensor(),
                                                                      transforms.Normalize((mean,), (std,))
                                                                      ])
                                       )

Triplet_test_dataset = Triplet_dataset(original_dataset=test_dataset.dataset,
                                       is_train = False,
                                        transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                                      transforms.Resize((28,28)),
                                                                      transforms.ToTensor(),
                                                                      transforms.Normalize((mean,), (std,))
                                                                      ])
                                       )

## Part 2: Network

### 2.1 Network Code

Embedding Network

In [5]:
import torch.nn as nn
import torch.nn.functional as F
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(1, 32, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

        self.fc = nn.Sequential(nn.Linear(64 * 4 * 4, 512),
                                nn.PReLU(),
#                                 nn.Linear(512, 512),
#                                 nn.PReLU(),
#                                 nn.Linear(512, 512)
                                )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)

Triplet Network

In [6]:
import torch.nn as nn
import torch.nn.functional as F
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

Triplet loss function

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()

### 2.2 Create Network

In [8]:
import torch.optim as optim
from torch.optim import lr_scheduler

batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(Triplet_train_dataset, batch_size=batch_size, shuffle=False, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(Triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

## Part 3: Train Network

### 3.1 Trainer class

In [11]:
import numpy as np
import random
import torch
import os
from PIL import Image

from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler


def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0):
    """
    Loaders, model, loss function and metrics should work together for a given task,
    i.e. The model should be able to process data output of loaders,
    loss function should process target output of loaders and outputs from the model

    Examples: Classification: batch loader, classification model, NLL loss, accuracy metric
    Siamese network: Siamese loader, siamese model, contrastive loss
    Online triplet learning: batch loader, embedding model, online triplet loss
    """
    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()

        # Train stage
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)

        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)

        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, n_epochs,
                                                                                 val_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        print(message)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()


        optimizer.zero_grad()
        outputs = model(*data)

        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses))
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message)
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            target = target if len(target) > 0 else None
            if not type(data) in (tuple, list):
                data = (data,)
            if cuda:
                data = tuple(d.cuda() for d in data)
                if target is not None:
                    target = target.cuda()

            outputs = model(*data)

            if type(outputs) not in (tuple, list):
                outputs = (outputs,)
            loss_inputs = outputs
            if target is not None:
                target = (target,)
                loss_inputs += target

            loss_outputs = loss_fn(*loss_inputs)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()

            for metric in metrics:
                metric(outputs, target, loss_outputs)

    return val_loss, metrics

### 3.2 Train Network

In [12]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 1/20. Train set: Average loss: 1.0000
Epoch: 1/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 2/20. Train set: Average loss: 1.0000
Epoch: 2/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 3/20. Train set: Average loss: 1.0000
Epoch: 3/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 4/20. Train set: Average loss: 1.0000
Epoch: 4/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 5/20. Train set: Average loss: 1.0000
Epoch: 5/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 6/20. Train set: Average loss: 1.0000
Epoch: 6/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 7/20. Train set: Average loss: 1.0000
Epoch: 7/20. Validation set: Average loss: 1.0000
Train: [0/837 (0%)]	Loss: 1.000000
Epoch: 8/20. Train set: Average loss: 1.0000
Epoch: 8/2

### 3.3 Save Network parameters

In [13]:
torch.save(model.state_dict(), "triplet_state.pth")

## Part 4: Extract Embedding

### 4.1 Extracter method

In [15]:
def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 512))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

### 4.2 Extract embedding and save to csv

In [16]:
import pandas as pd

train_embeddings_cl, train_labels_cl = extract_embeddings(train_loader, model)
train_embeddings_cl_df = pd.DataFrame(train_embeddings_cl)
train_embeddings_cl_df.to_csv("embedding_space_triplet.csv",index=False)

## Part 5: Save image path with index

In [17]:
image_list = train_dataset.dataset.imgs
image_list_df = pd.DataFrame(image_list)
image_list_df.to_csv("image_path_triplet.csv",index=False)

In [ ]:
# new_image_path = '/Users/y/Desktop/program1/input_pic1.jpg'
# image_list =  image_list_df.values.tolist()
# train_embeddings_cl = train_embeddings_cl_df.values.tolist()

In [ ]:
# mean, std = 0.1307, 0.3081
# transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
#                                                             transforms.Resize((28,28)),
#                                                             transforms.ToTensor(),
#                                                             transforms.Normalize((mean,), (std,))
#                                                             ])

In [ ]:
# image = Image.open(new_image_path)
# image = transform(image)
# image.unsqueeze_(0)
# new_picture_embedding = model.get_embedding(image).data.cpu().numpy()



In [ ]:
# distance = []
# I = []

# for i in train_embeddings_cl:
#     d = 0
#     for j in range(len(new_picture_embedding[0])):
#         ds = (new_picture_embedding[0][j] - i[j])**2
#         d = d + ds
#     distance.append(np.sqrt(d))

# for i in sorted(distance)[0:10]:
#     I.append(distance.index(i))

# path_list = []
# for i in I:
#     temp = image_list[i][0]
#     temp = temp.replace("\\", '/')
#     temp = 'program/' + temp
#     path_list.append(temp)

In [ ]:
# sorted(distance)

In [ ]:
# path_list

In [ ]:
# from PIL import Image
# a = Image.open(new_image_path)
# a

In [ ]:
# b = Image.open('/Users/y/Documents/TeamStaySafe/web/static/'+path_list[0])
# b